In [247]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

### Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [248]:
data = pd.read_csv('./features.csv', index_col='match_id')

# целевая переменная radiant_win - 1, если победила команда Radiant, 0 — иначе
labels = data['radiant_win'].values

# матрица признаков с удаленными столбцами итогов матчей
feature_matrix = data[data.columns[:102]].values

# обзор всех названий столбцов в features.csv
# print(data.columns.tolist())

data.head(10)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,4,4,0,-43.0,1453,0,512,2038,0,63
8,1430293701,1,8,3,967,1136,7,1,0,8,...,6,3,0,10.0,1968,0,1536,1983,12,63
9,1430299335,7,35,5,2117,1252,16,0,0,6,...,3,4,0,-15.0,4079,1,1540,0,63,0
11,1430308974,1,17,5,1527,906,10,0,1,7,...,3,4,0,26.0,3071,0,0,1572,0,63


2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [249]:
count_data = data.count()
print(type(count_data), '\n')

print('Count of rows = ', data.shape[0])
print('Count of columns = ', data.shape[1], '\n')

data_gaps = count_data[count_data != data.shape[0]]

print(data_gaps, '\n')
print('Count of columns with gaps = ', len(data_gaps))

<class 'pandas.core.series.Series'> 

Count of rows =  97230
Count of columns =  108 

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64 

Count of columns with gaps =  12


#### Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 принимают пропущенное значение.

По первым 4 строчкам понятно, что в 77677 матчах была пролита "первая кровь", причем в 53243 случаях в сражении столкнулись герои противоположных команд. Все остальные герои (77677 - 53243 = 24434), вполне вероятно, были убиты "крипами".

#### Если событие приобретения предмета "bottle", "courier", "flying_courier"  или установки командой первого "наблюдателя"  не успело произойти за первые 5 минут, то признаки времени принимают пропущенное значение.

Признаки:
    - `radiant_(dire_)bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_(dire_)courier_time`: время приобретения предмета "courier" 
    - `radiant_(dire_)flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_(dire_)first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
    
Относительно общего количества данных пропусков не так много (найденные пропуски не являются ошибкой в данных).

3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [250]:
data_new = pd.read_csv('./features.csv', index_col='match_id')

# замена пропусков на нули (предпочтительней для логистической регрессии)
data_new = data_new.fillna(0)
feature_matrix_without_nan = data_new[data_new.columns[:102]].values

# Check
# count_data_new = data_new.count()
# data_gaps_new = count_data_new[count_data_new != data.shape[0]]
# print(data_gaps_new, '\n')

4. Какой столбец содержит целевую переменную? Запишите его название.

   'radiant_win'

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

Время и качество настройки классификаторов на кросс-валидации представлено ниже в таблице:
<table style="width:70%">
<tr>
    <th></th>
    <th>N = 10</th>
    <th>N = 20</th>
    <th>N = 30</th>
    <th>N = 40</th>
</tr>

<tr>
    <td>alpha = 1.0</td>
    <td>Time elapsed: 0:01:17.939334
        Quality: 0.6839460613228764</td>
    <td>Time elapsed: 0:02:35.873202
        Quality: 0.6932450510364303</td>
    <td>Time elapsed: 0:03:46.897966
        Quality: 0.696246643884732</td>
    <td>Time elapsed: 0:04:59.107564
        Quality: 0.6994056410463603</td>
</tr>

<tr>
    <td>alpha = 0.6</td>
    <td>Time elapsed: 0:01:15.500730
        Quality: 0.6844870937566777</td>
    <td>Time elapsed: 0:02:30.116968
        Quality: 0.696756457319011</td>
    <td>Time elapsed: 0:03:51.717075
        Quality: 0.7012176221206963</td>
    <td>Time elapsed: 0:05:12.192088
        Quality: 0.7044702846923743</td>
</tr>
    
<tr>
    <td>alpha = 0.5</td>
    <td>Time elapsed: 0:01:16.144365
        Quality: 0.6855754043433915</td>
    <td>Time elapsed: 0:02:31.213344
        Quality: 0.6977219147013</td>
    <td>Time elapsed: 0:03:48.512043
        Quality: 0.7024742811365648</td>
    <td>Time elapsed: 0:04:51.417064
        Quality: 0.7053528845927085</td>
</tr>
    
<tr>
    <td>alpha = 0.4</td>
    <td>Time elapsed: 0:01:14.453331
        Quality: 0.6848478336673425 </td>
    <td>Time elapsed: 0:02:25.422657
        Quality: 0.6968689706745028</td>
    <td>Time elapsed: 0:03:37.766221
        Quality: 0.7020918864682237</td>
    <td>Time elapsed: 0:05:00.626732
        Quality: 0.7054686737895607</td>
</tr>
</table>

Оптимум не достигнут. С увеличением количества деревьев качество классификатора растет, так еще в 90-ых годах было доказано, что с увеличением числа базовых алгоритмов бустинг практически не переобучается. P.S. Переобучение может возникать, но после сотен, тысяч базовых алгоритмов.

In [120]:
kf = KFold(n_splits=5, shuffle=True, random_state=42) # кросс-валидация по 5 блокам; shuffle - перемешанная
n_tree = [10, 20, 30, 40] 
alpha_rate = [1, 0.6, 0.5, 0.4] # alpha = 0.5 is optimal
for n in n_tree:
    print('Count_of_trees = ', n, '\n')
    for alpha in alpha_rate:
        print('alpha = ', alpha)
        clf = GradientBoostingClassifier(learning_rate = alpha, n_estimators=n, verbose=False, random_state=241)
        start_time = datetime.datetime.now()
        sc = cross_val_score(estimator=clf, cv=kf, X=feature_matrix_without_nan, y=labels, scoring = "roc_auc")
        print('Time elapsed:', datetime.datetime.now() - start_time)
        print('Quality = ', np.mean(sc), '\n')

Count_of_trees =  10 

alpha =  1
Time elapsed: 0:01:17.939334
Quality =  0.6839460613228764 

alpha =  0.6
Time elapsed: 0:01:15.500730
Quality =  0.6844870937566777 

alpha =  0.5
Time elapsed: 0:01:16.144365
Quality =  0.6855754043433915 

alpha =  0.4
Time elapsed: 0:01:14.453331
Quality =  0.6848478336673425 

Count_of_trees =  20 

alpha =  1
Time elapsed: 0:02:35.873202
Quality =  0.6932450510364303 

alpha =  0.6
Time elapsed: 0:02:30.116968
Quality =  0.696756457319011 

alpha =  0.5
Time elapsed: 0:02:31.213344
Quality =  0.6977219147013 

alpha =  0.4
Time elapsed: 0:02:25.422657
Quality =  0.6968689706745028 

Count_of_trees =  30 

alpha =  1
Time elapsed: 0:03:46.897966
Quality =  0.696246643884732 

alpha =  0.6
Time elapsed: 0:03:51.717075
Quality =  0.7012176221206963 

alpha =  0.5
Time elapsed: 0:03:48.512043
Quality =  0.7024742811365648 

alpha =  0.4
Time elapsed: 0:03:37.766221
Quality =  0.7020918864682237 

Count_of_trees =  40 

alpha =  1
Time elapsed: 0:04:5

Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти выше по тексту. Какое качество при этом получилось?

In [121]:
kf = KFold(n_splits=5, shuffle=True, random_state=42) # кросс-валидация по 5 блокам; shuffle - перемешанная
n = 30 
alpha = 0.5 # alpha = 0.5 is optimal
print('Count_of_trees = ', n)
print('alpha = ', alpha)
clf = GradientBoostingClassifier(learning_rate = alpha, n_estimators=n, verbose=False, random_state=241)
start_time = datetime.datetime.now()
sc = cross_val_score(estimator=clf, cv=kf, X=feature_matrix_without_nan, y=labels, scoring = "roc_auc")
print('Time elapsed:', datetime.datetime.now() - start_time)
print('Quality = ', np.mean(sc), '\n')

Count_of_trees =  30
alpha =  0.5
Time elapsed: 0:03:50.530911
Quality =  0.7024742811365648 



### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

**Важно:** не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Лучший параметр регуляризации С = 0.01. Наилучшее качество логистической регрессии: 0.7165498862352037. Эта оценка примерно на 0.1 лучше качества градиентного бустинга при 30 деревьях. Качество градиентного бустинга можно увеличить за счет увеличения количества деревьев, но это неэффективно. Градиентный бустинг плохо работает с разреженными данными, в то время как линейные методы хорошо подходят для разреженных данных. Как было сказано ранее, линейные методы работают гораздо быстрее градиентного бустинга. В доказательство представлен код ниже.

In [124]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(feature_matrix_without_nan)

1 способ подбора лучшего параметра С c помощью GridSearchCV. По результатам: лучший параметр С = 0.01

In [148]:
# поиск лучшего параметра регуляризации
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 10000]
}

clf_1 = LogisticRegression(penalty = 'l2', solver='saga', random_state=241)

# создадим объект GridSearchCV
# cv = 5  (to use the default 5-fold cross validation)
search = GridSearchCV(clf_1, param_grid, cv=5, refit=True, scoring='roc_auc')


# запустим поиск
search.fit(X_train_scaled, labels)

# выведем наилучшие параметры
print(search.best_params_)

{'C': 0.01}


2 способ. Перепроверка 1 способа. Цикл for. По результатам: лучший параметр С = 0.01

In [150]:
kf = KFold(n_splits=5, shuffle=True, random_state=42) # кросс-валидация по 5 блокам; shuffle - перемешанная 
c_array = [0.0001, 0.001, 0.01, 0.1, 1, 10, 10000] 
for c in c_array:
    print('C = ', c)
    clf_2 = LogisticRegression(penalty = 'l2', random_state=241, C = c)
    start_time = datetime.datetime.now()
    sc = cross_val_score(estimator=clf_2, cv=kf, X=X_train_scaled, y=labels, scoring = "roc_auc")
    # проверяла без масшабирования с X = feature_matrix_without_nan - качество, действительно, на 0.2 хуже 
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print('Quality = ', np.mean(sc), '\n')

C =  0.0001
Time elapsed: 0:00:03.271122
Quality =  0.7113523612276411 

C =  0.001
Time elapsed: 0:00:05.161047
Quality =  0.7163630589772682 

C =  0.01
Time elapsed: 0:00:07.110924
Quality =  0.7165498862352037 

C =  0.1
Time elapsed: 0:00:07.714577
Quality =  0.7165269451581027 

C =  1
Time elapsed: 0:00:06.960011
Quality =  0.7165221087939807 

C =  10
Time elapsed: 0:00:07.691596
Quality =  0.7165218906414321 

C =  10000
Time elapsed: 0:00:07.669605
Quality =  0.7165218482557646 



2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

Лучший параметр регуляризации С = 0.01. Лучшее качество = 0.7165592000076536. Отличия есть, но качество практически не изменилось. Предположительно, вес удаленных признаков в исходной построенной модели был близок у нулю, поэтому они не оказывали большого влияния на прогноз.

In [175]:
data_new_features = pd.read_csv('./features.csv', index_col='match_id')

# замена пропусков на нули (предпочтительней для логистической регрессии)
data_new_features = data_new_features.fillna(0)

data_new_features.drop(['lobby_type','radiant_win','duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1, inplace=True)
for i in range(1, 6):
    del data_new_features['r{}_hero'.format(i)]
    del data_new_features['d{}_hero'.format(i)]
    
feature_matrix_without_nan_new = data_new_features.values

scaler_new = StandardScaler()

X_train_scaled_new = scaler_new.fit_transform(feature_matrix_without_nan_new)

data_new_features.head()
# после удаления 91 столбец = 108 - 6 - 11

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [176]:
kf = KFold(n_splits=5, shuffle=True, random_state=42) # кросс-валидация по 5 блокам; shuffle - перемешанная 
c_array = [0.0001, 0.001, 0.01, 0.1, 1, 10, 10000] 
for c in c_array:
    print('C = ', c)
    clf_2 = LogisticRegression(penalty = 'l2', random_state=241, C = c)
    start_time = datetime.datetime.now()
    sc = cross_val_score(estimator=clf_2, cv=kf, X=X_train_scaled_new, y=labels, scoring = "roc_auc") 
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print('Quality = ', np.mean(sc), '\n')

C =  0.0001
Time elapsed: 0:00:02.314205
Quality =  0.7113327497042613 

C =  0.001
Time elapsed: 0:00:03.807816
Quality =  0.7163758696833188 

C =  0.01
Time elapsed: 0:00:05.588363
Quality =  0.7165592000076536 

C =  0.1
Time elapsed: 0:00:05.734984
Quality =  0.7165338144534041 

C =  1
Time elapsed: 0:00:06.188820
Quality =  0.7165303443778955 

C =  10
Time elapsed: 0:00:05.913610
Quality =  0.7165305328738455 

C =  10000
Time elapsed: 0:00:05.933890
Quality =  0.7165304248419043 



3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [186]:
data_heroes = pd.read_csv("heroes.csv")
count_of_heroes = len(pd.unique(data_heroes["localized_name"].values))
print("Count_of_heroes = ", count_of_heroes)

Count_of_heroes =  112


4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [219]:
data_new_features_1 = pd.read_csv('./features.csv', index_col='match_id')

# замена пропусков на нули (предпочтительней для логистической регрессии)
data_new_features_1 = data_new_features_1.fillna(0)

data_new_features_1.drop(['lobby_type','radiant_win','duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1, inplace=True)

print(np.shape(data_new_features_1))
print(len(data_new_features_1.index.tolist()))

(97230, 101)
97230


In [220]:
X_pick = np.zeros((data_new_features_1.shape[0], count_of_heroes))

for i, match_id in enumerate(data_new_features_1.index):
    for p in range(5):
        X_pick[i, data_new_features_1.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_new_features_1.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick[0,:]

array([ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
        0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [243]:
data_new_features_1 = pd.read_csv('./features.csv', index_col='match_id')

# замена пропусков на нули (предпочтительней для логистической регрессии)
data_new_features_1 = data_new_features_1.fillna(0)

data_new_features_1.drop(['lobby_type','radiant_win','duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1, inplace=True)

for i in range(1, 6):
    del data_new_features_1['r{}_hero'.format(i)]
    del data_new_features_1['d{}_hero'.format(i)]
    
feature_matrix_with_wards_features = data_new_features_1.values

scaler_new_1 = StandardScaler()

X_train_scaled_new_1 = scaler_new_1.fit_transform(feature_matrix_with_wards_features)

print(np.shape(X_train_scaled_new_1), np.shape(X_pick))

feature_matrix = np.hstack((X_train_scaled_new_1, X_pick))

print(np.shape(feature_matrix))

(97230, 91) (97230, 112)
(97230, 203)


  5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?
   
C =  0.1

Time elapsed: 0:00:19.739499

Quality =  0.7519459356097022. 

Качество улучшилось, связано это с тем, что влияние на модель закодированных таким образом признаков стало более осмысленным. Матрица признаков стала более разреженной, что хорошо сказалось на качестве модели, построенной с использованием логистической регрессии. 

In [224]:
kf = KFold(n_splits=5, shuffle=True, random_state=42) # кросс-валидация по 5 блокам; shuffle - перемешанная 
c_array = [0.0001, 0.001, 0.01, 0.1, 1, 10, 10000] 
for c in c_array:
    print('C = ', c)
    clf_2 = LogisticRegression(penalty = 'l2', random_state=241, C = c)
    start_time = datetime.datetime.now()
    sc = cross_val_score(estimator=clf_2, cv=kf, X=feature_matrix, y=labels, scoring = "roc_auc", n_jobs=-1)
    # проверяла без масшабирования с X = feature_matrix_without_nan - качество, действительно, на 0.2 хуже 
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print('Quality = ', np.mean(sc), '\n')

C =  0.0001
Time elapsed: 0:00:07.561568
Quality =  0.7250378099266611 

C =  0.001
Time elapsed: 0:00:09.860658
Quality =  0.7463327741266581 

C =  0.01
Time elapsed: 0:00:16.967364
Quality =  0.7517386754874054 

C =  0.1
Time elapsed: 0:00:19.739499
Quality =  0.7519459356097022 

C =  1
Time elapsed: 0:00:19.079750
Quality =  0.7519271593364094 

C =  10
Time elapsed: 0:00:21.466455
Quality =  0.7519254976786709 

C =  10000
Time elapsed: 0:00:21.105399
Quality =  0.751923152300131 



6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [246]:
data_test = pd.read_csv('./features_test.csv', index_col='match_id')
data_test = data_test.fillna(0)
del data_test['lobby_type']
X_pick_test = np.zeros((data_test.shape[0], count_of_heroes))

for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick_test[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

for n in range(1, 6):
    del data_test['r{}_hero'.format(n)]
    del data_test['d{}_hero'.format(n)]

X_test_scaled = scaler_new_1.transform(data_test.values)

feature_test_matrix = np.hstack((X_test_scaled, X_pick_test))

clf_best = LogisticRegression(penalty = 'l2', random_state=241, C = 0.1, max_iter=150)
clf_best.fit(feature_matrix, labels)
pred = clf_best.predict_proba(feature_test_matrix)[:, 1]
print(pred)
print('max_pred = ', pred.max(), 'min_pred = ', pred.min())

[0.82393247 0.75300278 0.18807133 ... 0.23447581 0.62501815 0.42609193]
max_pred =  0.996372268518266 min_pred =  0.008425528529967617
